In [2]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Load the dataset and display the first 5 rows
df = pd.read_csv('./Loan_default.csv')
df.head()

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


The target feature for this dataset is 'Default' as it's the column with 0's and 1's as values due to this being a binary classification task.

# 1. Perform Exploratory Data Analysis (EDA)

### Number of rows and columns

In [4]:
rows, cols = df.shape
print(f'Rows: {rows}')
print(f'Columns: {cols}')

Rows: 255347
Columns: 18


### Data Types

In [5]:
print('Data Types: ')
df.dtypes

Data Types: 


LoanID             object
Age                 int64
Income              int64
LoanAmount          int64
CreditScore         int64
MonthsEmployed      int64
NumCreditLines      int64
InterestRate      float64
LoanTerm            int64
DTIRatio          float64
Education          object
EmploymentType     object
MaritalStatus      object
HasMortgage        object
HasDependents      object
LoanPurpose        object
HasCoSigner        object
Default             int64
dtype: object

There is a mostly even mix of numerical and categorical features. We plan to use one-hot encoding on the categorical features and scale the numerical features.

### Statistics for numerical columns

In [6]:
# Generate summary statistics and round them
summary_stats = df.describe().round(2)

# Convert the summary statistics into a DataFrame
summary_stats_df = pd.DataFrame(summary_stats)

summary_stats_df.drop(columns='Default')

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio
count,255347.00,255347.00,255347.00,255347.00,255347.00,255347.00,255347.00,255347.00,255347.00
mean,43.50,82499.30,127578.87,574.26,59.54,2.50,13.49,36.03,0.50
std,14.99,38963.01,70840.71,158.90,34.64,1.12,6.64,16.97,0.23
min,18.00,15000.00,5000.00,300.00,0.00,1.00,2.00,12.00,0.10
25%,31.00,48825.50,66156.00,437.00,30.00,2.00,7.77,24.00,0.30
50%,43.00,82466.00,127556.00,574.00,60.00,2.00,13.46,36.00,0.50
75%,56.00,116219.00,188985.00,712.00,90.00,3.00,19.25,48.00,0.70
max,69.00,149999.00,249999.00,849.00,119.00,4.00,25.00,60.00,0.90


### Statistics for categorical columns

In [7]:
# Create an empty list to store the analysis for each categorical column
categorical_analysis = []

# Loop through each categorical column and gather information
categorical_columns = df.select_dtypes(include=['object'])
categorical_columns = categorical_columns.drop(columns='LoanID')

for col in categorical_columns:
    # Get unique value count
    unique_values = df[col].nunique()
    
    # Get the mode (most frequent value)
    mode = df[col].mode()[0]
    mode_count = df[col].value_counts().iloc[0]
    mode_pct = (mode_count / len(df)) * 100

    # Get the second mode (second most frequent value if it exists)
    if len(df[col].value_counts()) > 1:
        second_mode = df[col].value_counts().index[1]
        second_mode_count = df[col].value_counts().iloc[1]
        second_mode_pct = (second_mode_count / len(df)) * 100
    else:
        second_mode = None
        second_mode_count = 0
        second_mode_pct = 0

    # Append the results to the list
    categorical_analysis.append({
        'Feature': col,
        'Unique Values': unique_values,
        'Mode': mode,
        'Mode Count': mode_count,
        'Mode %': mode_pct,
        'Second Mode': second_mode,
        'Second Mode Count': second_mode_count,
        'Second Mode %': second_mode_pct
    })

# Convert the list of dictionaries into a DataFrame
categorical_analysis_df = pd.DataFrame(categorical_analysis)

# Display the resulting DataFrame without index
categorical_analysis_df.style.hide()

Feature,Unique Values,Mode,Mode Count,Mode %,Second Mode,Second Mode Count,Second Mode %
Education,4,Bachelor's,64366,25.207267,High School,63903,25.025945
EmploymentType,4,Part-time,64161,25.126984,Unemployed,63824,24.995007
MaritalStatus,3,Married,85302,33.406306,Divorced,85033,33.300959
HasMortgage,2,Yes,127677,50.001371,No,127670,49.998629
HasDependents,2,Yes,127742,50.026826,No,127605,49.973174
LoanPurpose,5,Business,51298,20.089525,Home,51286,20.084826
HasCoSigner,2,Yes,127701,50.010770,No,127646,49.989230


In [8]:
def detect_outliers_iqr(df, column):
    """
    Detect outliers in a column using the IQR method.
    
    Parameters:
    df (DataFrame): The DataFrame containing the data.
    column (str): The column to check for outliers.
    
    Returns:
    DataFrame: A DataFrame containing the outliers for that column.
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR


    lower_outliers = df[df[column] < lower_bound]
    upper_outliers = df[df[column] > upper_bound]
    # outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    outliers = pd.concat([lower_outliers, upper_outliers], axis=0)
    
    return outliers, lower_outliers, upper_outliers, lower_bound, upper_bound


# Step 1: Select all numerical columns from the DataFrame
numerical_columns = df.select_dtypes(include=['float64', 'int64']).drop(columns='Default').columns

# Step 2: Detect outliers for each numerical column
outliers_dict = {}
for col in numerical_columns:
    outliers, *_ = detect_outliers_iqr(df, col)
    if not outliers.empty:  # Only store columns with outliers
        outliers_dict[col] = outliers

# Step 3: Display the outliers for each numerical column
if outliers_dict:
    print("Using IQR Rule we found outliers in:")
    for col, outliers in outliers_dict.items():
        print(f"  '{col}': {len(outliers)} entries")
else:
    print("No outliers detected in any numerical columns.")


No outliers detected in any numerical columns.


In [9]:
df.dtypes

LoanID             object
Age                 int64
Income              int64
LoanAmount          int64
CreditScore         int64
MonthsEmployed      int64
NumCreditLines      int64
InterestRate      float64
LoanTerm            int64
DTIRatio          float64
Education          object
EmploymentType     object
MaritalStatus      object
HasMortgage        object
HasDependents      object
LoanPurpose        object
HasCoSigner        object
Default             int64
dtype: object

In [10]:
# use one-hot encoding for categorical features
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

# get the categorical and numerical features
categorical_features = df.select_dtypes(include='object').columns.drop('LoanID')
numerical_features = df.select_dtypes(include='number').columns.drop('Default')

# initialize OrdinalEncoder
ordinal = OrdinalEncoder()

# ordinal encode the data --> returns a numpy array
ordinal_encoded = ordinal.fit_transform(df[categorical_features])

# make a DataFrame with the ordinal encoded data
ordinal_encoded_df = pd.DataFrame(ordinal_encoded, columns=categorical_features)

# concatenate with the original DataFrame after dropping the original unencoded categorical features to grab the numerical features
df_ordinal = pd.concat([df.drop(columns=categorical_features).reset_index(drop=True), ordinal_encoded_df.reset_index(drop=True)], axis=1)
df_ordinal = df_ordinal.drop(columns=['LoanID'])

# Initialize StandardScaler
scaler = StandardScaler()

df_ordinal[numerical_features] = scaler.fit_transform(df[numerical_features])

df_ordinal

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Default,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner
0,0.833990,0.089693,-1.086833,-0.341492,0.590533,1.341937,0.261771,-0.001526,-0.260753,0,0.0,0.0,0.0,1.0,1.0,4.0,1.0
1,1.701221,-0.823021,-0.044309,-0.731666,-1.285731,-1.343791,-1.308350,1.412793,0.778585,0,2.0,0.0,1.0,0.0,0.0,4.0,1.0
2,0.166888,0.043854,0.022715,-0.775718,-0.968209,0.446694,1.156831,-0.708685,-0.823728,1,2.0,3.0,0.0,1.0,1.0,0.0,0.0
3,-0.767053,-1.303452,-1.168538,1.061875,-1.718715,0.446694,-0.967805,-0.708685,-1.170174,0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1.100830,-1.592855,-1.671921,0.369631,-1.487790,1.341937,-1.052188,0.705634,0.995114,0,0.0,3.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255342,-1.634285,-1.142632,1.173101,-0.209337,1.427636,1.341937,0.093006,-1.415845,1.514783,0,0.0,0.0,1.0,0.0,0.0,4.0,0.0
255343,-0.767053,-0.783984,0.879724,-0.398130,-1.314597,-0.448549,-0.292744,-0.708685,-1.256785,1,1.0,1.0,0.0,0.0,0.0,3.0,0.0
255344,0.833990,0.059562,1.139391,0.143078,0.301877,0.446694,-1.236022,1.412793,-0.000918,0,1.0,2.0,1.0,1.0,1.0,0.0,1.0
255345,-0.099952,0.066979,-0.945840,1.477221,-0.564091,-1.343791,1.116146,0.705634,-0.260753,0,1.0,1.0,2.0,1.0,1.0,4.0,0.0


In [11]:
df_ordinal.to_csv('./preprocessed_data.csv')

Ordinal encoding was used for its simplicity compared to one-hot encoding. Furthermore, the accuracies for the baseline model trained using ordinal encoded data vs one-hot encoded data were identical, with feature selection for ordinal encoded data being significantly faster due to lower dimensionality.

In [12]:
from sklearn.model_selection import train_test_split

X = df_ordinal.drop(['Default'], axis=1)
y = df_ordinal['Default']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size=0.3,
                                                   shuffle=True,
                                                   stratify=y,
                                                   random_state=42)

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Train classifier using all features
model = DecisionTreeClassifier(max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model using accuracy
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.8849030742118661


# Feature selection methods

In [13]:
n_features = df_ordinal.shape[1] - 1 

## *k*-best features 

In [14]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from tqdm import tqdm

k_best_accuracy_list = []

for k in tqdm(range(1, n_features + 1), desc="K Best Progress"):
    k_best_selector = SelectKBest(mutual_info_classif, k=k)
    k_best_selector.fit(X_train, y_train)
    
    sel_X_train = k_best_selector.transform(X_train)
    sel_X_test = k_best_selector.transform(X_test)
    
    model.fit(sel_X_train, y_train)
    k_best_preds = model.predict(sel_X_test)
    
    # Calculate accuracy 
    accuracy = accuracy_score(y_test, k_best_preds)
    k_best_accuracy_list.append(accuracy)

print([round(acc, 3) for acc in k_best_accuracy_list])

ModuleNotFoundError: No module named 'tqdm'

In [15]:
def plot_accuracies(accuracies_list):
    x = np.arange(1, len(accuracies_list)+1)
    y = [round(acc, 3) for acc in accuracies_list]

    _, ax = plt.subplots(figsize=(8, 6))

    # Create the vertical bar plot
    bars = ax.bar(x, y, width=0.4, color='lightblue')

    # Set x and y labels
    ax.set_xlabel('Number of features selected')
    ax.set_ylabel('Accuracy')
    ax.set_ylim(0, 1.2)

    # Set custom ticks for x-axis
    ax.set_xticks(x)
    ax.set_xticklabels(np.arange(1, len(accuracies_list)+1))

    # Add value labels above the bars
    for bar, value in zip(bars, y):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.03, f'{value:.3f}', ha='center')

    plt.tight_layout()
    
def run_model_and_report(model, selector):
    model.fit(sel_X_train, y_train)
    preds = model.predict(sel_X_test)

    # Report selected featrures
    selected_feature_mask = selector.get_support()
    selected_features = X_train.columns[selected_feature_mask]
    print(f"Chosen features: {list(selected_features)}")

    # Calculate accuracy 
    accuracy = accuracy_score(y_test, preds)
    print(f"Accuracy = {accuracy}")

In [16]:
plot_accuracies(accuracies_list=k_best_accuracy_list)

NameError: name 'k_best_accuracy_list' is not defined

In [17]:
# Model with the best number of features according to k_best_selector
k_best_selector = SelectKBest(mutual_info_classif, k=16)
k_best_selector.fit(X_train, y_train)
sel_X_train = k_best_selector.transform(X_train)
sel_X_test = k_best_selector.transform(X_test)

run_model_and_report(model=model,selector=k_best_selector)

Chosen features: ['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio', 'Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner']
Accuracy = 0.8849030742118661


## Recursive feature elimination (RFE) 

In [18]:
from sklearn.feature_selection import RFE

rfe_accuracy_list = []

# Use tqdm to track the progress of the loop
for k in tqdm(range(1, n_features + 1), desc="RFE Progress"):
    RFE_selector = RFE(estimator=model, n_features_to_select=k, step=1)
    RFE_selector.fit(X_train, y_train)
    
    sel_X_train = RFE_selector.transform(X_train)
    sel_X_test = RFE_selector.transform(X_test)
    
    model.fit(sel_X_train, y_train)
    RFE_preds = model.predict(sel_X_test)

    accuracy = model.score(sel_X_test, y_test)
    rfe_accuracy_list.append(accuracy)


NameError: name 'tqdm' is not defined

In [ ]:
plot_accuracies(accuracies_list=rfe_accuracy_list[0:15])

In [ ]:
# Model with the best number of features according to RFE_selector
RFE_selector = RFE(estimator=model, n_features_to_select=10, step=1)
RFE_selector.fit(X_train, y_train)
sel_X_train = RFE_selector.transform(X_train)
sel_X_test = RFE_selector.transform(X_test)

run_model_and_report(model=model,selector=RFE_selector)

Both k-best and RFE have comparable accuracies to the baseline model. Therefore, we will be using those feature selection techiniques.

## Variance threshold

In [ ]:
# Calculate the variance of each feature
X_train.var(axis=0)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train)

In [ ]:
# Plot sorted variances
x = X.columns
y = scaled_X_train.var(axis=0)

# Sort the features and variance values
sorted_indices = np.argsort(y)[::-1]  # Sorting in descending order
sorted_x = x[sorted_indices]          # Sorted feature names
sorted_y = y[sorted_indices]          # Sorted variances

# Create horizontal bar plot
plt.figure(figsize=(10, 6))
bars = plt.barh(sorted_x, sorted_y)

# Add value labels next to the bars
for bar, value in zip(bars, sorted_y):
    plt.text(bar.get_width() + 0.001, bar.get_y() + bar.get_height()/2, f'{value:.3f}', va='center')

# Adjust x-axis limit to prevent clipping
plt.xlim(0, max(sorted_y) * 1.1)

# Remove x-ticks
plt.xticks([])

# Set labels and title
plt.xlabel('Variance')
plt.title('Feature Variances')

# Invert y-axis to have the most variance at the top
plt.gca().invert_yaxis()

plt.tight_layout()


In [ ]:
X_train_dropped = X_train.drop(['DTIRatio', 'Income', 'InterestRate'], axis=1)
X_test_dropped = X_test.drop(['DTIRatio', 'Income', 'InterestRate'], axis=1)

model.fit(X_train_dropped, y_train)

# Make predictions
preds = model.predict(X_test_dropped)

# Evaluate the model using accuracy
accuracy = accuracy_score(y_test, preds)
print(accuracy)

Variance threshold has the lowest accuracy. We're not going to use it.

In [ ]:
#Plotting the Decision Tree
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

ordinal = OrdinalEncoder()

# Define features and target variable
X = df.drop('Default', axis=1)
y = df['Default']

#Ordinal Transform the 
for x in df.columns:
    if df.column == "object":
        df[df.column] = ordinal.fit_transform[df.column]
        
#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#Train the Decision Tree Classifier
classifier = DecisionTreeClassifier(max_depth=10, min_samples_split=20, min_samples_leaf=2)

classifier.fit(X_train, y_train)  # Train the model
y_pred = classifier.predict(X_test)  # Make predictions

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

plt.figure(figsize=(12, 8))
plot_tree(classifier, filled=True, feature_names=X.columns, max_depth=10)
plt.title("Decision Tree (First 4 Levels)")
plt.show()

In [ ]:
#Tuning the Hyperparameters for the Decision Tree
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Define the model
dt = DecisionTreeClassifier(random_state=42)

# Set the parameter grid
dt_grid = {
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5, 10],
    'max_features': ['auto'],
    'criterion': ['entropy'],
    'max_leaf_nodes': [None, 5, 10, 20]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=dt, param_grid=dt_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)